In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
spark = SparkSession.builder.master("local[30]").appName("graphlab_amazon_fraud.com").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
review_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/reviews_Musical_Instruments.json")
meta_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/meta_Musical_Instruments.json")

In [ ]:
review_df.count()

In [ ]:
review_df.select(F.count_distinct("reviewerID"))

In [ ]:
equals_zero = lambda e: e == 0
rev_df = review_df.withColumn("drop", F.exists(F.col("helpful"), equals_zero))
rev_df = rev_df.filter(F.col("drop") == False).drop(F.col("drop"))
rev_df = rev_df.select("asin", "overall", "reviewText", "reviewerID", "reviewerName", "summary", rev_df.helpful[0], rev_df.helpful[1], "unixReviewTime", "reviewTime")
rev_df

In [ ]:
# rev_df = rev_df.groupBy("reviewerID").agg(F.sum("helpful[0]").alias("helpful_0"), \
#                                 F.sum("helpful[1]").alias("helpful_1"), \
#                                 F.collect_list("asin").alias("asin"), \
#                                 F.collect_list("overall").alias("overall"), \
#                                 F.collect_list("reviewText").alias("reviewText"), \
#                                 F.collect_list("summary").alias("summary"), \
#                                 F.collect_list("reviewerName").alias("reviewerName"), \
#                                 F.collect_list("unixReviewTime").alias("unixReviewTime"), \
#                                 F.collect_list("reviewTime").alias("reviewTime")
#                                 )

In [8]:
rev_df = rev_df.withColumn("label", (F.col("helpful[0]") / F.col("helpful[1]"))).drop("helpful[0]").drop("helpful[1]")
rev_df = rev_df.filter((F.col("label") > 0.8) | (F.col("label") < 0.2))
rev_df = rev_df.withColumn("label", F.udf(lambda a: 1 if a > 0.8 else 0, T.IntegerType())(F.col("label")))

In [9]:
rev_df

asin,overall,reviewText,reviewerID,reviewerName,summary,unixReviewTime,reviewTime,label
0634029347,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,1129334400,"10 15, 2005",0
0634029347,5.0,If you are an una...,A16WE7UU0QD33D,S. R. Mcdonald,Solo Years Revisited,1128211200,"10 2, 2005",0
0634029355,2.0,I was carefull to...,AXMWZYP2IROMP,Alex,not great instruc...,1352419200,"11 9, 2012",1
0634029355,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,1129334400,"10 15, 2005",0
0634029355,5.0,Unless you are so...,A6DCKXX4659CR,"Hotshot ""Hotshot""",CLAPTON GUITAR IN...,1189728000,"09 14, 2007",1
0634029355,5.0,I've played guita...,A28YJZCV43ZWQW,James V. Signorile,First rate lesson...,1206489600,"03 26, 2008",1
0634029363,2.0,As much as I like...,A1M7OWNI93N1Y3,geronimo,not very didactic,1365897600,"04 14, 2013",1
0634029363,5.0,For those who may...,A338HVUM4AWJY7,Joaquim Do M. Nt ...,This is an instru...,1182038400,"06 17, 2007",1
0634029363,1.0,Very disappointin...,AKKQM6B0KI6F4,Thomas J. Chuchola,Clapton Classics,1272672000,"05 1, 2010",1
0634055534,5.0,"So, you have a pi...",A175UOHOVXRYMW,"Diane E. Amov ""Th...",Frustrated No More!,1214092800,"06 22, 2008",1


In [10]:
rev_df.count()

152391

In [ ]:
# rev_df = rev_df.withColumn("reviewLen", F.udf(lambda a: len(a), T.IntegerType())(F.col("reviewerName")))
# rev_df = rev_df.withColumn("reviewSetLen", F.udf(lambda a: len(set(a)), T.IntegerType())(F.col("reviewerName")))
# rev_df = rev_df.drop(F.col("reviewLen")).drop(F.col("reviewSetLen"))

In [ ]:
reviewers_set = set([row[0] for row in rev_df.select('reviewerID').collect()])

In [ ]:
len(reviewers_set)

In [ ]:
# rev_df_json = rev_df.toJSON()
rev_df.coalesce(1).write.format('json').save("/mnt/raid0_ssd_8tb/dataset/Amazon/Refined/Musical_Instruments")

In [ ]:
rev_df.printSchema()

In [ ]:
# id_list = list(rev_df.select('reviewerID').toPandas()['reviewerID'])

In [ ]:
# s = set()
# dup = list()
# for r_id in id_list:
#     if r_id in s:
#         dup.append(r_id)
#     else:
#         s.add(r_id)

In [ ]:
# print(dup)

In [ ]:
# rev_df.filter(rev_df.reviewerID == 'A1R8OFZY6CYX7G').show()

In [ ]:
rev_df.select(count_distinct("reviewerID")).show()

In [ ]:
rev_df.printSchema()

In [ ]:
meta_df.printSchema()

In [ ]:
meta_df

In [ ]:
meta_df.count()

In [ ]:
upu_groupings = rev_df.groupBy("asin").agg(F.collect_list("reviewerID").alias("reviewerID"))

In [ ]:
upu_groupings

In [ ]:
# upu_groupings = upu_groupings.select("asin", "reviewerID").filter((F.col('reviewerID').isin(reviewers_set)))

In [ ]:
upu_groupings.count()

In [ ]:
def format_date(review_time):
    month, day, year = review_time.replace(",","").split(" ")
    return f"{int(month):02d}-{int(day):02d}-{int(year):04d}"

In [ ]:
format_date_udf = F.udf(lambda x: format_date(x), T.StringType())
usu_groupings = rev_df.withColumn("formatted_date", F.to_date(format_date_udf(F.concat_ws(",", "reviewTime")), "MM-dd-yyyy"))
usu_groupings = usu_groupings.groupBy(F.weekofyear("formatted_date").alias("week"), "overall").agg(F.collect_list("reviewerID").alias("reviewerID"))#.collect()

In [ ]:
usu_groupings.count()

In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [12]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
words_df = tokenizer.transform(rev_df)

In [13]:
hashingTF = HashingTF(inputCol="words", outputCol="tf_features")
tf_df = hashingTF.transform(words_df)

In [14]:
# featurizedData

In [15]:
idf = IDF(inputCol="tf_features", outputCol="features")
idfModel = idf.fit(tf_df)
tfidf_df = idfModel.transform(tf_df)

In [16]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features", outputCol="norm")
tfidf_df= normalizer.transform(tfidf_df).drop(F.col("tf_features")).drop(F.col("features"))

In [17]:
tfidf_df

22/09/27 09:18:56 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
22/09/27 09:18:56 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


asin,overall,reviewText,reviewerID,reviewerName,summary,unixReviewTime,reviewTime,label,words,norm
0634029347,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,1129334400,"10 15, 2005",0,"[i, bought, this,...","(262144,[11104,11..."
0634029347,5.0,If you are an una...,A16WE7UU0QD33D,S. R. Mcdonald,Solo Years Revisited,1128211200,"10 2, 2005",0,"[if, you, are, an...","(262144,[19036,30..."
0634029355,2.0,I was carefull to...,AXMWZYP2IROMP,Alex,not great instruc...,1352419200,"11 9, 2012",1,"[i, was, carefull...","(262144,[3834,694..."
0634029355,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,1129334400,"10 15, 2005",0,"[i, bought, this,...","(262144,[11568,19..."
0634029355,5.0,Unless you are so...,A6DCKXX4659CR,"Hotshot ""Hotshot""",CLAPTON GUITAR IN...,1189728000,"09 14, 2007",1,"[unless, you, are...","(262144,[2701,392..."
0634029355,5.0,I've played guita...,A28YJZCV43ZWQW,James V. Signorile,First rate lesson...,1206489600,"03 26, 2008",1,"[i've, played, gu...","(262144,[161,1546..."
0634029363,2.0,As much as I like...,A1M7OWNI93N1Y3,geronimo,not very didactic,1365897600,"04 14, 2013",1,"[as, much, as, i,...","(262144,[921,5798..."
0634029363,5.0,For those who may...,A338HVUM4AWJY7,Joaquim Do M. Nt ...,This is an instru...,1182038400,"06 17, 2007",1,"[for, those, who,...","(262144,[150,2306..."
0634029363,1.0,Very disappointin...,AKKQM6B0KI6F4,Thomas J. Chuchola,Clapton Classics,1272672000,"05 1, 2010",1,"[very, disappoint...","(262144,[6946,190..."
0634055534,5.0,"So, you have a pi...",A175UOHOVXRYMW,"Diane E. Amov ""Th...",Frustrated No More!,1214092800,"06 22, 2008",1,"[so,, you, have, ...","(262144,[150,3091..."


In [18]:
tfidf_sample_df = spark.createDataFrame(tfidf_df.head(10))
# tfidf_sample_df = tfidf_df
# tfidf_sample_df = tfidf_df.sample(fraction=0.3, seed=3)

22/09/27 09:18:56 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


In [19]:
tfidf_sample_df.count()

10

In [20]:
# features = [row[0] for row in tfidf_df.select('norm').collect()]
# features

In [21]:
dot_prd = F.udf(lambda x,y: float(x.dot(y)), T.DoubleType())
tfidf_dot_df = tfidf_sample_df.alias("i").join(tfidf_sample_df.alias("j"), F.col("i.norm") < F.col("j.norm"))\
    .select(
        F.col("i.reviewerID").alias("i_reviewerID"),
        F.col("j.reviewerID").alias("j_reviewerID"),
        F.col("i.norm").alias("i"), 
        F.col("j.norm").alias("j"), 
        dot_prd("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")

# dot_prd = F.udf(lambda x,y: float(x.dot(y)), T.DoubleType())
# tfidf_sample_df.alias("i").join(tfidf_sample_df.alias("j"), F.col("i.norm") < F.col("j.norm"))\
#     .select(
#         F.col("i.norm").alias("i"), 
#         F.col("j.norm").alias("j"), 
#         dot_prd("i.norm", "j.norm").alias("dot"))\
#     .sort("i", "j")

In [22]:
tfidf_dot_df = tfidf_dot_df.drop(F.col("i")).drop(F.col("j"))

In [23]:
tfidf_dot_df = tfidf_dot_df.groupBy("i_reviewerID").agg(F.collect_list("j_reviewerID").alias("j_reviewerID"), \
                                F.collect_list("dot").alias("dot")
                                )

In [24]:
tfidf_dot_df.count()

8

In [25]:
# topn = F.udf(lambda a: zip(*heapq.nlargest(5, enumerate(a), key=operator.itemgetter(1)))[0], T.ArrayType(T.IntegerType()))
topn = F.udf(lambda a: sorted(range(len(a)), key=lambda i: a[i])[-5:], T.ArrayType(T.IntegerType()))
tfidf_dot_df = tfidf_dot_df.withColumn("dot", topn(F.col('dot')))
# metadata_tags_df = metadata_tags_df.withColumn('input_ids', F.udf(lambda z: PLMinput_function(z), pst.ArrayType(pst.IntegerType()))(F.col('Synopsis')))

In [26]:
tfidf_dot_df

i_reviewerID,j_reviewerID,dot
A28YJZCV43ZWQW,"[A1M7OWNI93N1Y3, ...","[3, 0, 4, 1, 5]"
A6DCKXX4659CR,"[AXMWZYP2IROMP, A...","[4, 0, 1, 2, 3]"
A175UOHOVXRYMW,"[A28YJZCV43ZWQW, ...","[7, 1, 3, 2, 0]"
A1M7OWNI93N1Y3,"[A6DCKXX4659CR, A...","[4, 5, 0, 1, 2]"
A338HVUM4AWJY7,"[A175UOHOVXRYMW, ...","[5, 1, 6, 7, 3]"
A1MUVHT8BONL5K,"[A1MUVHT8BONL5K, ...","[2, 1, 0]"
AXMWZYP2IROMP,"[AKKQM6B0KI6F4, A...","[3, 0, 1, 2]"
AKKQM6B0KI6F4,"[A1MUVHT8BONL5K, ...","[2, 0, 1]"


In [29]:
extract_index = F.udf(lambda a, b: [b[i] for i in a], T.ArrayType(T.StringType()))
uvu_grouping = tfidf_dot_df.withColumn("j_reviewerID", extract_index(F.col('dot'), F.col('j_reviewerID'))).drop(F.col("dot"))

In [30]:
uvu_grouping.collect()

[Row(i_reviewerID='A28YJZCV43ZWQW', j_reviewerID=['AKKQM6B0KI6F4', 'A1M7OWNI93N1Y3', 'A1MUVHT8BONL5K', 'A6DCKXX4659CR', 'A1MUVHT8BONL5K']),
 Row(i_reviewerID='A6DCKXX4659CR', j_reviewerID=['A16WE7UU0QD33D', 'AXMWZYP2IROMP', 'AKKQM6B0KI6F4', 'A1MUVHT8BONL5K', 'A1MUVHT8BONL5K']),
 Row(i_reviewerID='A175UOHOVXRYMW', j_reviewerID=['A16WE7UU0QD33D', 'A1M7OWNI93N1Y3', 'AXMWZYP2IROMP', 'A6DCKXX4659CR', 'A28YJZCV43ZWQW']),
 Row(i_reviewerID='A1M7OWNI93N1Y3', j_reviewerID=['A1MUVHT8BONL5K', 'A16WE7UU0QD33D', 'A6DCKXX4659CR', 'AXMWZYP2IROMP', 'AKKQM6B0KI6F4']),
 Row(i_reviewerID='A338HVUM4AWJY7', j_reviewerID=['AKKQM6B0KI6F4', 'A28YJZCV43ZWQW', 'A1MUVHT8BONL5K', 'A1MUVHT8BONL5K', 'A6DCKXX4659CR']),
 Row(i_reviewerID='A1MUVHT8BONL5K', j_reviewerID=['A16WE7UU0QD33D', 'A16WE7UU0QD33D', 'A1MUVHT8BONL5K']),
 Row(i_reviewerID='AXMWZYP2IROMP', j_reviewerID=['A16WE7UU0QD33D', 'AKKQM6B0KI6F4', 'A1MUVHT8BONL5K', 'A1MUVHT8BONL5K']),
 Row(i_reviewerID='AKKQM6B0KI6F4', j_reviewerID=['A16WE7UU0QD33D', 'A1MUVH

In [ ]:
# tfidf_dot_sample_df = spark.createDataFrame(tfidf_dot_df.head(10))

In [ ]:
# tfidf_dot_sample_df

In [ ]:
# result = data.alias("i").join(data.alias("j"), F.col("i.ID") < F.col("j.ID"))\
#     .select(
#         F.col("i.ID").alias("i"),
#         F.col("j.ID").alias("j"),
#         sim_cos("i.feature", "j.feature").alias("sim_cosine"))\
#     .sort("i", "j")
# result.show()

In [ ]:
tfidf_dot_df.write.mode('Overwrite').json("/mnt/raid0_ssd_8tb/dataset/Amazon/tfidf_dot_df")